##### Importacao de bibliotecas e banco de dados

In [10]:
import pandas as pd
from pathlib import Path
import numpy as np

In [11]:
base_path = Path("database")
base_airbnb = pd.DataFrame()
for file in base_path.iterdir():
    if file.name.endswith(".csv"):
        month_df = pd.read_csv(base_path / file.name)
        month_df['last_scraped'] = pd.to_datetime(month_df['last_scraped'])
        month_df['year'] = month_df['last_scraped'].dt.year
        month_df['month'] = month_df['last_scraped'].dt.month
        base_airbnb = pd.concat([base_airbnb, month_df], ignore_index=True)


/tmp/ipykernel_284973/3276474334.py:5: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  month_df = pd.read_csv(base_path / file.name)
/tmp/ipykernel_284973/3276474334.py:5: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  month_df = pd.read_csv(base_path / file.name)
/tmp/ipykernel_284973/3276474334.py:5: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  month_df = pd.read_csv(base_path / file.name)
/tmp/ipykernel_284973/3276474334.py:5: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  month_df = pd.read_csv(base_path / file.name)
/tmp/ipykernel_284973/3276474334.py:5: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  month_df = pd.read_csv(base_path / file.name)
/tmp/ipykernel_284973/3276474334.py:

##### Limpeza do dataset

Como tempo muitas colunas no nosso dataset, o modelo fica mais lento.

Uma análise será feita para retirar as colunas que não serão necessárias para o nosso modelo de previsao.

colunas que serão excluídas:

- IDs, links e informações nao relevantes ao modelo.
- criada colunas separadas para ano e mes de extracao (mais util para a análise), excluindo a coluna com a data completa
- Colunas preenchidas com texto livre (nao será utilizado nenhuma análise de palavras, ou LLM, focando-se um modelo mais simples)
- colunas com todos (ou proximo a isso) são iguais
- precos que nao sejam diarios
- informacoes fiscais exigidas (análise busca verificar características do imovel)

In [12]:
print(list(base_airbnb.columns))

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price', '

In [13]:
columns = [
    "host_response_time",
    "host_response_rate",
    "host_is_superhost",
    "host_listings_count",
    "latitude",
    "longitude",
    "property_type",
    "room_type",
    "accommodates",
    "bathrooms",
    "bedrooms",
    "beds",
    "bed_type",
    "amenities",
    "price",
    "security_deposit",
    "cleaning_fee",
    "guests_included",
    "extra_people",
    "minimum_nights",
    "maximum_nights",
    "number_of_reviews",
    "review_scores_rating",
    "review_scores_accuracy",
    "review_scores_cleanliness",
    "review_scores_checkin",
    "review_scores_communication",
    "review_scores_location",
    "review_scores_value",
    "instant_bookable",
    "is_business_travel_ready",
    "cancellation_policy",
    "year",
    "month",
]
base_airbnb = base_airbnb.loc[:, columns]
display(base_airbnb)

,host_response_time,host_response_rate,host_is_superhost,host_listings_count,latitude,longitude,property_type,room_type,accommodates,bathrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,year,month
0,within an hour,100%,t,2.0,-22.965920,-43.178960,Condominium,Entire home/apt,5,1.0,...,10.0,10.0,10.0,10.0,9.0,t,f,strict_14_with_grace_period,2019,11
1,within a day,91%,f,3.0,-22.977120,-43.190450,Apartment,Entire home/apt,2,1.0,...,10.0,9.0,10.0,10.0,9.0,f,f,strict_14_with_grace_period,2019,11
2,within an hour,100%,t,1.0,-22.983020,-43.214270,Apartment,Entire home/apt,3,1.0,...,10.0,10.0,10.0,10.0,10.0,t,f,strict_14_with_grace_period,2019,11
3,within an hour,100%,t,1.0,-22.988160,-43.193590,Apartment,Entire home/apt,3,1.5,...,9.0,10.0,10.0,10.0,9.0,f,f,strict_14_with_grace_period,2019,11
4,within an hour,100%,t,1.0,-22.981270,-43.190460,Loft,Entire home/apt,2,1.0,...,10.0,10.0,10.0,10.0,10.0,f,f,strict_14_with_grace_period,2019,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902205,within an hour,100%,t,22.0,-22.982284,-43.190167,Apartment,Entire home/apt,4,1.5,...,NaN,NaN,NaN,NaN,NaN,t,f,strict_14_with_grace_period,2018,7
902206,within a day,80%,f,2.0,-23.019163,-43.488247,Serviced apartment,Private room,2,1.0,...,NaN,NaN,NaN,NaN,NaN,t,f,moderate,2018,7
902207,within a day,60%,f,19.0,-22.964826,-43.177641,Apartment,Entire home/apt,4,1.0,...,NaN,NaN,NaN,NaN,NaN,t,f,flexible,2018,7
902208,NaN,NaN,f,1.0,-22.890793,-43.440360,House,Entire home/apt,7,2.0,...,NaN,NaN,NaN,NaN,NaN,t,f,flexible,2018,7


#### Tratando dados Faltantes

Visualizando os dados, percebemos que existem uma grande disparidade em dados faltantes.
- as colunas comm mais de 300 mil valores nulos foram excluidas da análise
- Para as demais colunas, como temos mais de 900 mil linhas, vamos simplesmente exlcuir com os dados nulos

In [14]:
print(base_airbnb.isnull().sum())
for column in base_airbnb:
    if base_airbnb[column].isnull().sum() > 300000:
        base_airbnb = base_airbnb.drop(column, axis=1)
print(base_airbnb.shape)

host_response_time             401843
host_response_rate             401846
host_is_superhost                 460
host_listings_count               460
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                        1724
bedrooms                          850
beds                             2502
bed_type                            0
amenities                           0
price                               0
security_deposit               421280
cleaning_fee                   313506
guests_included                     0
extra_people                        0
minimum_nights                      0
maximum_nights                      0
number_of_reviews                   0
review_scores_rating           448016
review_scores_accuracy         448586
review_scores_cleanliness      448413
review_scores_checkin          448650
review_score

In [15]:
base_airbnb = base_airbnb.dropna()
print(base_airbnb.isnull().sum())
print(base_airbnb.shape)

host_is_superhost           0
host_listings_count         0
latitude                    0
longitude                   0
property_type               0
room_type                   0
accommodates                0
bathrooms                   0
bedrooms                    0
beds                        0
bed_type                    0
amenities                   0
price                       0
guests_included             0
extra_people                0
minimum_nights              0
maximum_nights              0
number_of_reviews           0
instant_bookable            0
is_business_travel_ready    0
cancellation_policy         0
year                        0
month                       0
dtype: int64
(897709, 23)


In [16]:
print(base_airbnb.dtypes)
print("-"*60)
print(base_airbnb.iloc[0])

host_is_superhost            object
host_listings_count         float64
latitude                    float64
longitude                   float64
property_type                object
room_type                    object
accommodates                  int64
bathrooms                   float64
bedrooms                    float64
beds                        float64
bed_type                     object
amenities                    object
price                        object
guests_included               int64
extra_people                 object
minimum_nights                int64
maximum_nights                int64
number_of_reviews             int64
instant_bookable             object
is_business_travel_ready     object
cancellation_policy          object
year                          int32
month                         int32
dtype: object
------------------------------------------------------------
host_is_superhost                                                           t
host_listings_count

Como preço e extra_people estão sendo reconhecidos como objeto ao ines de float, tempo de mudar o tipo de variável da coluna

In [17]:
# alterar price, extra_people
base_airbnb["price"] = base_airbnb["price"].str.replace("$", "").str.replace(",", "")
base_airbnb["price"] = base_airbnb["price"].astype(np.float32, copy=False)
base_airbnb["extra_people"] = base_airbnb["extra_people"].str.replace("$", "").str.replace(",", "")
base_airbnb["extra_people"] = base_airbnb["extra_people"].astype(np.float32, copy=False)
print(base_airbnb.dtypes)

host_is_superhost            object
host_listings_count         float64
latitude                    float64
longitude                   float64
property_type                object
room_type                    object
accommodates                  int64
bathrooms                   float64
bedrooms                    float64
beds                        float64
bed_type                     object
amenities                    object
price                       float32
guests_included               int64
extra_people                float32
minimum_nights                int64
maximum_nights                int64
number_of_reviews             int64
instant_bookable             object
is_business_travel_ready     object
cancellation_policy          object
year                          int32
month                         int32
dtype: object


#### Análise Exploratória e Tratamento de Outliers

Analisando caracteristica por caracteristica, iremos:

- Ver a correlação entre as features e decidir se manteremos todas as features que temos
- Exluir outliers (usaremos como regra: valores abaixo de Q1 - 1,5 x Amplitude e valores acima de Q3 + 1,5 x Amplitude) onde Amplitude = Q3 - Q1
- Confirmar se todas as features que temos realmente fazem sentido para o nosso modelo ou se alguma delas nao irá nos ajudar ou se devemos excluí-la